In [1]:
import mbuild as mb
from cmeutils.sampling import is_equilibrated, equil_sample
import signac
import gsd.hoomd
import numpy as np
import glob
import MDAnalysis as mda
from MDAnalysis.analysis import polymer
import matplotlib.pyplot as plt

project = signac.get_project()
project.find_jobs()


/home/madilynpaul/miniconda3/envs/cmeutils/lib/python3.11/site-packages/mdtraj/formats/__init__.py:6: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  from .xtc import XTCTrajectoryFile
/home/madilynpaul/miniconda3/envs/cmeutils/lib/python3.11/site-packages/mdtraj/geometry/order.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import parse_version
/home/madilynpaul/miniconda3/envs/cmeutils/lib/python3.11/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/home/madilynpaul/miniconda3/envs/cmeutils/lib/python3.11/site-packages/mbuild/recipes/__init_

,sp.num_mols,sp.lengths,sp.density,sp.ti_ratio,sp.remove_hydrogens,sp.remove_charges,sp.sigma_scale,sp.kT,sp.n_steps,sp.shrink_kT,...,doc.ref_mass_units,doc.ref_energy,doc.ref_energy_units,doc.ref_length,doc.ref_length_units,doc.dt,doc.target_box,doc.tau_kT,doc.real_time_step,doc.real_time_units
b2a11e9f460770ae5d173afd3d132a7b,1,10,0.05,1.0,True,True,1.0,4.0,50000000.0,8.0,...,amu,1.046,kJ/mol,0.356359,nm,0.001,"[1.8481125381459392e-06, 1.8481125381459392e-0...",0.1,1.972896,fs
2363379bbf4a53eb408900a93bfb7838,1,10,0.05,1.0,True,True,1.0,8.0,50000000.0,8.0,...,amu,1.046,kJ/mol,0.356359,nm,0.001,"[1.8481125381459392e-06, 1.8481125381459392e-0...",0.1,1.972896,fs


In [52]:
u = mda.Universe('workspace/2363379bbf4a53eb408900a93bfb7838/cg-traj1.gsd')
traj = u.trajectory[0]

In [57]:
"""rewrite atom indices"""
bond_indices = []
particle_index = [0]
for i in range(len(u.bonds)):
    a = u.bonds[i].atoms.indices
    bond_indices.append(list(a))
    if particle_index[-1] in bond_indices[i]:
        atom1 = bond_indices[i][0]
        atom2 = bond_indices[i][1]
        if atom1 not in particle_index:
            particle_index.append(atom1)
        if atom2 not in particle_index:
            particle_index.append(atom2)


In [74]:
"""create bonds list"""
particle_positions = []
bonds = []
for i in particle_index:
    pos = u.atoms.positions[i]
    particle_positions.append(pos)
for i in range(len(u.bonds)):
    b = particle_positions[i+1]-particle_positions[i]
    bonds.append(b)

for i in range(len(bonds)-1):
    b1 = bonds[i]
    b2 = bonds[i+1]
    dot_product = np.dot(b1,b2)
    